# III) Création d'une ontologie

Pour ce faire, nous utiliserons le package `owlready2` qui permet de créer des ontologies en python. (https://linuxfr.org/news/owlready-un-module-python-pour-manipuler-les-ontologies-owl)
Les données seront extraite de la base de données de WikiData, nous interrogerons cette base de données par l'API mis à disposition par la librairie `wikibaseintegrator` (https://github.com/LeMyst/WikibaseIntegrator#execute-sparql-queries).

Commençons par installer les packages nécessaires

In [110]:
# !pip install owlready2
# !pip install wikibaseintegrator

Récupérons les classes et sous-classes de la classe Animalia du fichier `animals_id_name_subClass_sub2Class.csv` que nous avons créé précédemment. Nous avons décité de se limiter à deux sous-classes pour des raisons de temps de récupération des informations de l'API.
De plus, nous rencontrons un problème quand les sous classes ne sont pas définies dans WikiData, nous avons donc décidé de ne pas les prendre en compte, et de créer une sous-classe d'animalia appelé sans famille.

In [129]:
import pandas as pd

df = pd.read_csv("animals_id_name_subClass_sub2Class.csv")

tableau_animaux = df['animalLabel'].values.tolist()

# Sélectionner les colonnes spécifiques
selected_columns = ["subclass2Label", "subclassLabel"]
selected_data = df[selected_columns]

# Transformer les colonnes en un tableau de la forme [[subclass2Label], [subclassLabel]]
tableau = selected_data.values.T.tolist()

# Afficher le résultat
print(tableau_animaux)

['0', '0', '0', '0', '0', '0', '0', '0', 'lion', '0', '0', '0', '0', '0', '0', 'African elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'cheetah', '0', '0', '0', '0', 'leopard', 'cougar', 'jaguar', 'brown bear', '0', '0', 'electric ray', '0', '0', '0', '0', '0', 'broccoli', '0', '0', '0', 'Indian elephant', '0', '0', '0', '0', '0', '0']


Concernant la contrainte de la propriété possède pour les sous-classes d'Animalia, c'est un concept avancé dans OWL appelé "Restrictions sur les propriétés". Pour répondre au mieux à cette contrainte, nous allons utiliser AllDisjoint et AllDifferent.

In [169]:
import numpy as np
from owlready2 import Thing, AllDisjoint
import owlready2 as owl

onto = owl.get_ontology("http://example65.com/animalia.owl")

# Définir la classe racine Animalia
with onto:
    class Animalia(Thing):
        pass


# Définir les sous-classes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        for j in range(len(tableau_sans_doublons)):
            if i == 0: # cela dépend de Animalia
                nom_classe = tableau_sans_doublons[j].replace(" ", "_") # Remplacer les espaces par des underscores
                # Créer une classe dynamiquement
                classe = type(nom_classe, (Animalia,), {})
            else: # c'est une sous-classe du précédent
                indices_ssClass = [i for i, x in enumerate(tableau[i]) if x == tableau_sans_doublons[j]] # Récupérer les indices des sous-sous-classes
                if len(indices_ssClass) == 1: # Alors on crée sans réfléchir la sous classe
                    nom_sous_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
                    nom_classe = tableau[i-1][indices_ssClass[0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                    # Créer une classe dynamiquement
                    classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})
                else: # Alors on crée une classe en vérifiant s'il y a des doublons
                    suppresion_doublons = []
                    for k in range(len(indices_ssClass)):
                        suppresion_doublons.append([indices_ssClass[k], tableau[i][indices_ssClass[k]]])
                    uniques = [sublist for i, sublist in enumerate(suppresion_doublons) if sublist[1] not in [x[1] for x in suppresion_doublons[:i]]]
                    for l in range(len(uniques)):
                        nom_sous_classe = uniques[l][1].replace(" ", "_")  # Remplacer les espaces par des underscores
                        nom_classe = tableau[i-1][uniques[l][0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                        # Créer une classe dynamiquement
                        classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})

# Définir les classes orphelines
with onto:
    class SansFamille(Animalia):
        pass

# Définir les exclusions, toutes les classes d'un même niveau sont disjointes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        # Créer une liste de classes
        liste_classes = []
        for j in range(len(tableau_sans_doublons)):
            nom_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
            liste_classes.append(getattr(onto, nom_classe))
        # Créer une liste de classes disjointes
        AllDisjoint(liste_classes)

# Définir les différentes propriétés
with onto:
    class CaracteristiqueMorphologique(Thing):
        pass

    class possede(owl.ObjectProperty):
        domain    = [Animalia]
        range     = [CaracteristiqueMorphologique]
        pass

    class Patte(CaracteristiqueMorphologique):
        pass

    class Museau(CaracteristiqueMorphologique):
        pass

    class Aile(CaracteristiqueMorphologique):
        pass

    class Bec(CaracteristiqueMorphologique):
        pass

    class Queue(CaracteristiqueMorphologique):
        pass

    class Poil(CaracteristiqueMorphologique):
        pass

    class Plume(CaracteristiqueMorphologique):
        pass

On ajoute les animaux de la base de données de WikiData dans notre ontologie.

In [170]:
from owlready2 import get_ontology

onto = get_ontology("./FF3_animalia.owl").load()

for i in range(len(tableau[-1])):
    if tableau[-1][i] == '0':
        nom_animal = tableau[0][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        animal = SansFamille(nom_animal)
    else:
        nom_classe = tableau[-1][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        nom_animal = tableau_animaux[i].replace(" ", "_")  # Remplacer les espaces par des underscores
        print(nom_classe, nom_animal)
        # Accédez à la classe spécifique
        classe = onto[nom_classe]
        print(classe)
        # Créez une nouvelle instance de cette classe
        nouvel_animal = classe("nom_animal")


big_cat lion
animalia.big_cat
elephant African_elephant
animalia.elephant
big_cat cheetah
animalia.big_cat
big_cat leopard
animalia.big_cat
big_cat cougar
animalia.big_cat
big_cat jaguar
animalia.big_cat
bear brown_bear
animalia.bear
electric_fish electric_ray
animalia.electric_fish
vegetable broccoli
animalia.vegetable
Asian_elephant Indian_elephant
animalia.Asian_elephant


On en registre notre ontologie dans un fichier .owl

In [171]:
onto.save(file="FF3_animalia.owl", format="rdfxml")